In [160]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, RepeatVector, Concatenate, Dense, TimeDistributed
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import random
from transformers import MarianMTModel, MarianTokenizer

In [161]:
# Load and preprocess data
url = '/kaggle/input/eng-french/eng_-french.csv' 
data = pd.read_csv(url)
data.rename(columns={"English words/sentences": "Eng", "French words/sentences": "Frn"}, inplace=True)
french_sentences = data['Frn'].tolist()


In [162]:
def tokenize_and_pad(texts, max_len=None):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, padding='post', maxlen=max_len)
    return padded_sequences, tokenizer

eng_sequences, eng_tokenizer = tokenize_and_pad(data['Eng'])
frn_sequences, frn_tokenizer = tokenize_and_pad(data['Frn'])

eng_max_len = max(len(seq) for seq in eng_sequences)
frn_max_len = max(len(seq) for seq in frn_sequences)

eng_sequences, eng_tokenizer = tokenize_and_pad(data['Eng'], max_len=eng_max_len)
frn_sequences, frn_tokenizer = tokenize_and_pad(data['Frn'], max_len=frn_max_len)

eng_vocab_size = len(eng_tokenizer.word_index) + 1
frn_vocab_size = len(frn_tokenizer.word_index) + 1

train_eng, test_eng, train_frn, test_frn = train_test_split(eng_sequences, frn_sequences, test_size=0.2, random_state=42)



In [168]:
def define_seq2seq_model(eng_vocab_size, frn_vocab_size, eng_max_len, frn_max_len, embedding_dim=128, lstm_units=256):
    # Encoder
    encoder_inputs = Input(shape=(eng_max_len,))
    encoder_embedding = Embedding(eng_vocab_size, embedding_dim, input_length=eng_max_len, mask_zero=True)(encoder_inputs)
    
    # Bidirectional LSTM layer with return_state=True
    encoder_lstm = LSTM(units=512, return_sequences=True, return_state=True, dropout=0.5, recurrent_dropout=0.5, use_bias=True, kernel_initializer='glorot_uniform')(encoder_inputs)
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)
    
    # Concatenate forward and backward hidden states
    context_vector = Concatenate()([forward_h, backward_h])  # Shape: (batch_size, lstm_units * 2)
    
    # Repeat the context vector for each timestep in the decoder
    decoder_inputs = RepeatVector(frn_max_len)(context_vector)  # Shape: (batch_size, frn_max_len, lstm_units * 2)

    # Decoder
    decoder_lstm = LSTM(lstm_units * 2, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs)
    decoder_dense = TimeDistributed(Dense(frn_vocab_size, activation='softmax'))
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model
    model = Model(encoder_inputs, decoder_outputs)
    
    # Compile the model
    initial_learning_rate = 0.1
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=100000,
        decay_rate=0.5,
        staircase=True
    )
    
    model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model, encoder_inputs, encoder_outputs, forward_h, forward_c, backward_h, backward_c

In [169]:
embedding_dim = 128
lstm_units = 256
batch_size = 256
epochs = 3

model = define_seq2seq_model(eng_vocab_size, frn_vocab_size, eng_max_len, frn_max_len, embedding_dim, lstm_units)

# Prepare data for the decoder
train_frn_input = train_frn[:, :-1]  # input 
train_frn_output = train_frn[:, 1:]  # expected output 

# Padding train_frn_input to ensure it matches the required input shape
train_frn_input = pad_sequences(train_frn_input, maxlen=frn_max_len, padding='post')
train_frn_output = pad_sequences(train_frn_output, maxlen=frn_max_len, padding='post')

# Train the model
history = model.fit(train_eng, train_frn_output,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

# Plotting the loss and accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')

plt.show()

ValueError: Input 0 of layer "lstm_86" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 44)

In [156]:
# Define parameters
eng_max_len = 20
frn_max_len = 20
eng_vocab_size = 37000
frn_vocab_size = 37000
embedding_dim = 256
hidden_units = 512

# Define the encoder
encoder_inputs = Input(shape=(eng_max_len,))
encoder_embedding = Embedding(input_dim=eng_vocab_size, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
encoder_outputs, forward_h, forward_c = encoder_lstm(encoder_embedding)
encoder_model = Model(encoder_inputs, [encoder_outputs, forward_h, forward_c])

# Assuming you define your decoder LSTM with return states
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=[forward_h, forward_c])
decoder_dense = Dense(frn_vocab_size, activation='softmax')
decoder_final_output = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs, forward_h, forward_c], [decoder_final_output, state_h, state_c])

# Tokenizer preparation
def prepare_tokenizer(texts, special_tokens=None):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    
    if special_tokens:
        for token in special_tokens:
            tokenizer.word_index[token] = len(tokenizer.word_index) + 1
            tokenizer.index_word[len(tokenizer.word_index) + 1] = token
    
    return tokenizer

# Prepare French tokenizer
special_tokens = {'<start>': len(frn_tokenizer.word_index) + 1,
                   '<end>': len(frn_tokenizer.word_index) + 2}
frn_tokenizer = prepare_tokenizer(data['Frn'], special_tokens=special_tokens)

print("Shape of encoder input data:", train_frn_input.shape)
print("Shape of decoder input data:", train_frn_output.shape)

encoder_model.summary()
decoder_model.summary()


Shape of encoder input data: (140496, 55)
Shape of decoder input data: (140496, 55)


Model: "functional_133"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_66 (InputLayer)     │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_41 (Embedding)        │ (None, 20, 256)        │     9,472,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_83 (LSTM)                  │ [(None, 20, 512),      │     1,574,912 │
│                                 │ (None, 512), (None,    │               │
│                                 │ 512)]                  │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,046,912 (42.14 MB)

 Trainable params: 11,046,912 (42.14 MB)

...

Model: "functional_135"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_65      │ (None, 1, 37000)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ keras_tensor_393CL… │ [(None, 20, 512), │          0 │ -                 │
│ (InputLayer)        │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_84 (LSTM)      │ [(None, 1, 512),  │ 76,826,624 │ input_layer_65[4… │
│                     │ (None, 512),      │            │ keras_tensor_393… │
│                     │ (None, 512)]      │            │ keras_tensor_393… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_33 (Dense)    │ (None, 1, 37000)  │ 18,981,000 │ lstm_84[2][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 95,807,624 (365.48 MB)

 Trainable params: 95,807,624 (365.48 MB)

 Non-trainable params: 0 (0.00 B)

In [157]:
def preprocess_input_sequence(sentence, tokenizer, max_len):
    seq = [tokenizer.word_index.get(word, 0) for word in sentence.split()]
    seq = seq[:max_len] + [0] * (max_len - len(seq))
    return np.array(seq).reshape((1, max_len))

def decode_sequence(input_seq, encoder_model, decoder_model, frn_tokenizer, frn_max_len):
    # Get encoder states
    _, state_h, state_c = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, frn_vocab_size))
    target_seq[0, 0, frn_tokenizer.word_index['<start>']] = 1

    decoded_sentence = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq, state_h, state_c])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = frn_tokenizer.index_word.get(sampled_token_index, '')

        if sampled_token == '<end>' or len(decoded_sentence) > frn_max_len:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_token
            target_seq = np.zeros((1, 1, frn_vocab_size))
            target_seq[0, 0, sampled_token_index] = 1  # Prepare next input to decoder

            # Update the states
            state_h, state_c = h, c

    return decoded_sentence.strip()


In [159]:
# Translation function
def translate(sentence, encoder_model, decoder_model, eng_tokenizer, frn_tokenizer, eng_max_len, frn_max_len):
    input_seq = preprocess_input_sequence(sentence, eng_tokenizer, eng_max_len)
    return decode_sequence(input_seq, encoder_model, decoder_model, frn_tokenizer, frn_max_len)

# Function to translate and print random sentences
def translate_and_print_random_sentences(data, encoder_model, decoder_model, eng_tokenizer, frn_tokenizer, eng_max_len, frn_max_len, num_sentences=20):
    translations = []
    
    for _ in range(num_sentences):
        random_index = random.randint(0, len(data) - 1)
        original_sentence = data['Eng'].iloc[random_index]
        input_seq = preprocess_input_sequence(original_sentence, eng_tokenizer, eng_max_len)
        translated_sentence = decode_sequence(input_seq, encoder_model, decoder_model, frn_tokenizer, frn_max_len)
        
        translations.append({
            'Original Sentence': original_sentence,
            'Translated Sentence': translated_sentence
        })
    
    df_translations = pd.DataFrame(translations)
    print(df_translations)

# Example usage
translate_and_print_random_sentences(data, encoder_model, decoder_model, eng_tokenizer, frn_tokenizer, eng_max_len, frn_max_len)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━